Baseline: Predction using latest link

1. load data
2. iterate over data chunks
    1. store current chunk's links as dictionary (update dict)
    2. make prediction to evaluate
    3. store eval #

- First batch is not used for evaluation.
- For positive samples, if edge not found, considered as wrong answer (False Negative).
- For negative samples, if edge not found, considered as correct answer (True Negative).

In [1]:
!pwd

/c/Users/990924/OneDrive/code/nbs


In [3]:
import os
os.chdir(DATASET_DIR)
!pwd

/c/Users/990924/OneDrive/code/GNN_TGN_V3/data/DataV3


In [18]:
import numpy as np
import pandas as pd

In [29]:
DATASET_DIR = "C:/Users/990924/OneDrive/code/GNN_TGN_V3/data/DataV3"
NEG_SAMPLES_DIR = os.path.join(DATASET_DIR, "Results\One session diff dataset test\ForEach\L3_D5_20220619_PM")

In [5]:
session_dirs = [x for x in os.listdir(".") if "_D" in x]
session_dirs

['L1_D1_20220621_PM',
 'L1_D1_20220626_AM',
 'L1_D1_20220626_PM',
 'L1_D3_20220626_AM',
 'L1_D3_20220626_PM',
 'L2_D2_20220620_PM',
 'L2_D2_20220622_PM',
 'L2_D2_20220624_PM',
 'L2_D4_20220622_PM',
 'L2_D4_20220624_PM',
 'L3_D5_20220619_AM',
 'L3_D5_20220619_PM',
 'L3_D5_20220623_PM',
 'L3_D7_20220619_AM',
 'L3_D7_20220619_PM',
 'L3_D7_20220623_PM',
 'L4_D6_20220627_PM',
 'L4_D6_20220630_PM',
 'L4_D8_20220627_PM',
 'L4_D8_20220629_PM',
 'L4_D8_20220630_PM']

In [6]:
for session_dir in session_dirs[:1]:
    raw_dir = os.path.join(session_dir, "feat-desc_non-negative", "raw")
    csv_path = os.path.join(raw_dir, os.listdir(raw_dir)[0])
    df = pd.read_csv(csv_path)

In [7]:
df.head()

,recording,second,id,gaze_id,talking_id,angle_w,role_description,talking_description,position_description
0,L1_D1_20220621_PM,0.00,7,4,[],1.0,Gazing Student and gazed Student.,No one.,Gazing sitting opposite to being gazed.
1,L1_D1_20220621_PM,0.01,7,4,[],1.0,Gazing Student and gazed Student.,No one.,Gazing sitting opposite to being gazed.
2,L1_D1_20220621_PM,0.02,7,4,[],1.0,Gazing Student and gazed Student.,No one.,Gazing sitting opposite to being gazed.
3,L1_D1_20220621_PM,0.05,7,4,[],1.0,Gazing Student and gazed Student.,No one.,Gazing sitting opposite to being gazed.
4,L1_D1_20220621_PM,0.06,7,4,[],1.0,Gazing Student and gazed Student.,No one.,Gazing sitting opposite to being gazed.


In [8]:
df.describe()

,second,id,gaze_id,angle_w
count,155754.000000,155754.000000,155754.000000,155754.000000
mean,564.586805,6.620491,5.654442,0.737073
std,345.927742,1.472271,1.394351,0.239038
min,0.000000,4.000000,4.000000,0.250000
25%,258.662500,5.000000,4.000000,0.750000
50%,563.770000,7.000000,6.000000,0.750000
75%,838.470000,8.000000,6.000000,1.000000
max,1208.940000,8.000000,8.000000,1.000000


## Testing Positive Samples Only

In [27]:
# vers. acculumated edges

def baseline_latest_accum_pos(df, batch_size=100):
    edge_dict = {}
    
    res_correct = 0
    res_wrong = 0
    
    for iter in range(len(df)//batch_size):
            # update edge_dict
            for idx, row in df[iter*batch_size:(iter+1)*batch_size].iterrows():
                edge_dict[row["id"]] = row["gaze_id"]
    
            # predict for next batch
            for idx, row in df[(iter+1)*batch_size:(iter+2)*batch_size].iterrows():
                try:
                    if row["gaze_id"] == edge_dict[row["id"]]:
                        res_correct += 1
                    else:
                        res_wrong += 1
                except:
                    # if no dict recorded yet, set as wrong
                    res_wrong += 1

    return {
        "# correct": res_correct,
        "# wrong": res_wrong,
        "Accuracy": res_correct/(res_correct+res_wrong),
    }
            

In [24]:
for session_dir in session_dirs:
    raw_dir = os.path.join(session_dir, "feat-desc_non-negative", "raw")
    csv_path = os.path.join(raw_dir, os.listdir(raw_dir)[0])
    df = pd.read_csv(csv_path)

    print(session_dir)
    res = baseline_latest_accum_pos(df, batch_size=100)
    print(res)
    print("---")

L1_D1_20220621_PM
{'# correct': 130216, '# wrong': 25438, 'Accuracy': 0.8365734256748943}
---
L1_D1_20220626_AM
{'# correct': 185290, '# wrong': 35940, 'Accuracy': 0.8375446368033268}
---
L1_D1_20220626_PM
{'# correct': 219059, '# wrong': 7488, 'Accuracy': 0.9669472559777883}
---
L1_D3_20220626_AM
{'# correct': 153852, '# wrong': 9889, 'Accuracy': 0.9396058409317153}
---
L1_D3_20220626_PM
{'# correct': 138657, '# wrong': 10749, 'Accuracy': 0.9280550981888278}
---
L2_D2_20220620_PM
{'# correct': 109037, '# wrong': 6600, 'Accuracy': 0.9429248423947353}
---
L2_D2_20220622_PM
{'# correct': 67048, '# wrong': 7670, 'Accuracy': 0.897347359404695}
---
L2_D2_20220624_PM
{'# correct': 67337, '# wrong': 6206, 'Accuracy': 0.9156139945338102}
---
L2_D4_20220622_PM
{'# correct': 354402, '# wrong': 14878, 'Accuracy': 0.9597107885615251}
---
L2_D4_20220624_PM
{'# correct': 308208, '# wrong': 11554, 'Accuracy': 0.9638668759890169}
---
L3_D5_20220619_AM
{'# correct': 237591, '# wrong': 22340, 'Accuracy'

In [28]:
for session_dir in session_dirs:
    raw_dir = os.path.join(session_dir, "feat-desc_non-negative", "raw")
    csv_path = os.path.join(raw_dir, os.listdir(raw_dir)[0])
    df = pd.read_csv(csv_path)

    print(session_dir)
    res = baseline_latest_accum_pos(df, batch_size=1000)
    print(res)
    print("---")

L1_D1_20220621_PM
{'# correct': 82928, '# wrong': 71826, 'Accuracy': 0.535869832120656}
---
L1_D1_20220626_AM
{'# correct': 119352, '# wrong': 100978, 'Accuracy': 0.541696546089956}
---
L1_D1_20220626_PM
{'# correct': 200975, '# wrong': 24672, 'Accuracy': 0.8906610768146707}
---
L1_D3_20220626_AM
{'# correct': 135639, '# wrong': 27202, 'Accuracy': 0.8329536173322444}
---
L1_D3_20220626_PM
{'# correct': 122306, '# wrong': 26200, 'Accuracy': 0.8235761518053143}
---
L2_D2_20220620_PM
{'# correct': 98759, '# wrong': 15978, 'Accuracy': 0.8607423934737705}
---
L2_D2_20220622_PM
{'# correct': 54432, '# wrong': 19386, 'Accuracy': 0.737381126554499}
---
L2_D2_20220624_PM
{'# correct': 52600, '# wrong': 20043, 'Accuracy': 0.724089038172983}
---
L2_D4_20220622_PM
{'# correct': 315880, '# wrong': 52500, 'Accuracy': 0.8574841196590477}
---
L2_D4_20220624_PM
{'# correct': 275858, '# wrong': 43004, 'Accuracy': 0.8651328788002334}
---
L3_D5_20220619_AM
{'# correct': 195356, '# wrong': 63675, 'Accuracy

## Testing Positive and Negative Samples

In [41]:
def get_negative_samples(session_dir, batch_size):
    neg_samples_path = os.path.join(NEG_SAMPLES_DIR, f"Batch_Size_{batch_size}", "With Features", f"L3_D5_20220619_PM-M_{session_dir}.csv")
    df_neg = pd.read_csv(neg_samples_path)

    return df_neg

In [53]:
# vers. acculumated edges, neg samples

def baseline_latest_accum_posneg(df, df_neg, batch_size=100):
    edge_dict = {}
    
    res_tp = 0
    res_tn = 0
    res_fp = 0
    res_fn = 0
    
    for iter in range(len(df)//batch_size):
            # update edge_dict
            for idx, row in df[iter*batch_size:(iter+1)*batch_size].iterrows():
                edge_dict[row["id"]] = row["gaze_id"]
    
            # predict for next pos batch
            for idx, row in df_neg[(2*(iter+1))*batch_size:((2*(iter+1)+1)*batch_size)].iterrows():
                try:
                    if row["dst"] == edge_dict[row["src"]]:
                        res_tp += 1
                    else:
                        res_fn += 1
                except:
                    # if no dict recorded yet, set as wrong
                    res_fn += 1

            # predict for next neg batch
            for idx, row in df_neg[(2*(iter+1)+1)*batch_size:((2*(iter+1)+2)*batch_size)].iterrows():
                try:
                    if row["dst"] == edge_dict[row["src"]]:
                        res_fp += 1
                    else:
                        res_tn += 1
                except:
                    res_tn += 1
            # print(edge_dict)

    return {
        "tp": res_tp,
        "tn": res_tn,
        "fp": res_fp,
        "fn": res_fn,
        "F1": (2*res_tp)/((2*res_tp)+res_fp+res_fn)
    }
            

In [55]:
batch_size = 100

for session_dir in session_dirs:
    raw_dir = os.path.join(session_dir, "feat-desc_non-negative", "raw")
    csv_path = os.path.join(raw_dir, os.listdir(raw_dir)[0])
    df = pd.read_csv(csv_path)

    print(session_dir)
    df_neg = get_negative_samples(session_dir, batch_size=batch_size)
    res = baseline_latest_accum_posneg(df, df_neg, batch_size=batch_size)
    print(res)
    print("---")

L1_D1_20220621_PM
{'tp': 130226, 'tn': 145590, 'fp': 10018, 'fn': 25474, 'F1': 0.8800719054956343}
---
L1_D1_20220626_AM
{'tp': 185290, 'tn': 208206, 'fp': 12994, 'fn': 35940, 'F1': 0.8833555018426084}
---
L1_D1_20220626_PM
{'tp': 219078, 'tn': 222423, 'fp': 4077, 'fn': 7510, 'F1': 0.9742363972313076}
---
L1_D3_20220626_AM
{'tp': 153859, 'tn': 158276, 'fp': 5424, 'fn': 9905, 'F1': 0.9525487003439128}
---
L1_D3_20220626_PM
{'tp': 138657, 'tn': 143537, 'fp': 5863, 'fn': 10749, 'F1': 0.9434823731143213}
---
L2_D2_20220620_PM
{'tp': 109046, 'tn': 109016, 'fp': 6584, 'fn': 6602, 'F1': 0.9429863627322962}
---
L2_D2_20220622_PM
{'tp': 67048, 'tn': 67030, 'fp': 7670, 'fn': 7670, 'F1': 0.897347359404695}
---
L2_D2_20220624_PM
{'tp': 67347, 'tn': 67334, 'fp': 6166, 'fn': 6225, 'F1': 0.9157561953972193}
---
L2_D4_20220622_PM
{'tp': 354403, 'tn': 365329, 'fp': 3931, 'fn': 14897, 'F1': 0.974124353727286}
---
L2_D4_20220624_PM
{'tp': 308208, 'tn': 315564, 'fp': 4160, 'fn': 11592, 'F1': 0.97508257298

In [56]:
batch_size = 1000

for session_dir in session_dirs:
    raw_dir = os.path.join(session_dir, "feat-desc_non-negative", "raw")
    csv_path = os.path.join(raw_dir, os.listdir(raw_dir)[0])
    df = pd.read_csv(csv_path)

    print(session_dir)
    df_neg = get_negative_samples(session_dir, batch_size=batch_size)
    res = baseline_latest_accum_posneg(df, df_neg, batch_size=batch_size)
    print(res)
    print("---")

L1_D1_20220621_PM
{'tp': 82989, 'tn': 130412, 'fp': 24096, 'fn': 72011, 'F1': 0.6332983574031326}
---
L1_D1_20220626_AM
{'tp': 119352, 'tn': 186014, 'fp': 33986, 'fn': 100978, 'F1': 0.6388130640033398}
---
L1_D1_20220626_PM
{'tp': 201018, 'tn': 212864, 'fp': 12430, 'fn': 24982, 'F1': 0.9148659227030275}
---
L1_D3_20220626_AM
{'tp': 135655, 'tn': 148992, 'fp': 13690, 'fn': 27345, 'F1': 0.8686228369271158}
---
L1_D3_20220626_PM
{'tp': 122338, 'tn': 134361, 'fp': 13651, 'fn': 26662, 'F1': 0.8585454175424314}
---
L2_D2_20220620_PM
{'tp': 98800, 'tn': 98537, 'fp': 15937, 'fn': 16200, 'F1': 0.8601139563936153}
---
L2_D2_20220622_PM
{'tp': 54504, 'tn': 54322, 'fp': 19314, 'fn': 19496, 'F1': 0.7374474015343192}
---
L2_D2_20220624_PM
{'tp': 52807, 'tn': 52450, 'fp': 19836, 'fn': 20193, 'F1': 0.7251567188261708}
---
L2_D4_20220622_PM
{'tp': 315893, 'tn': 354888, 'fp': 13112, 'fn': 52627, 'F1': 0.9057539156302642}
---
L2_D4_20220624_PM
{'tp': 275861, 'tn': 304348, 'fp': 14376, 'fn': 43139, 'F1': 